## Map Object Data to Pickles

- author : Sylvie Dagoret-Campagne
- creation date : 2024-09-05
- last update : 2024-09-05

Goal generate a csv file that maps object number over Pickle filename from NearestNeighbourg in (G-R, Z-Y) color plane 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [ ]:
from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

In [ ]:
def FindNearestNeighbors(X_data,X_neigh):
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X_neigh)
    distances, indices = nbrs.kneighbors(X_data)
    return indices,distances  

In [ ]:
def FindNearestKDT(X_data,X_neigh):
    kdt = KDTree(X_neigh,leaf_size=30, metric='euclidean')
    distances,indices = kdt.query(X_data, k=1, return_distance=True)
    return indices,distances 

In [ ]:
def FindNearestBallT(X_data,X_neigh):
    bt = BallTree(X_neigh, metric='euclidean')
    distances,indices = bt.query(X_data,k=1,return_distance=True)
    return indices,distances  

In [ ]:
machine_name = os.uname().nodename
path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py310")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

In [ ]:
# reference flux in Jy
F0 = ((0.*u.ABmag).to(u.Jy)).value
F0


## Initialisation

In [ ]:
filename_catalogmatching = "catalog_match_objtopickles.csv" 

### Read the list of objects

In [ ]:
#filename_data =  "list_of_isolatedobjects.csv" # 67 objects, produced by buggy **MultiColor_lightCurves-DMRehearsal2024_01-AuxTel.ipynb**
filename_data = "isolated_objects_magandcolors_psfap_good.csv" # Must use this file produced by **ColorIsolatedObjects.ipynb**

In [ ]:
df_data = pd.read_csv(filename_data ,index_col=0)

In [ ]:
df_data

In [ ]:
def ComputeErrors(row):
    """
    """
    s_GR = np.sqrt(row["smagG"]**2 + row["smagR"]**2)
    s_ZY = np.sqrt(row["smagZ"]**2 + row["smagY"]**2)
    s_ap_GR = np.sqrt(row["sapmagG"]**2 + row["sapmagR"]**2)
    s_ap_ZY = np.sqrt(row["sapmagZ"]**2 + row["sapmagY"]**2)
    return pd.Series(data = [s_GR,s_ZY,s_ap_GR,s_ap_ZY],index= ["s_GR","s_ZY","s_ap_GR","s_ap_ZY"])

In [ ]:
if filename_data ==  "list_of_isolatedobjects.csv":
    df_data_err = df_data.apply(ComputeErrors,axis=1,result_type='expand')
    df_data_comb = df_data.join(df_data_err)
    df_data =   df_data_comb

### Atmosphere

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul = ObsAtmo("AUXTEL")

### My tool to process atmosphere

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections_auxtel import *

In [ ]:
def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed

In [ ]:
def ComputeColors(the_pc,the_sed1,the_sed2): 
    """
    parameters:
     the_pc : PhotometricCorrections object with multi-values PWV
     the_sed1 : the rubin_sim SED for object 1
     the_sed2 : another rubin_sim SED for object 2
     
    return:
     all_df_magcol : pandas dataframe with calculated colors G-R and Z-Y for the two objects 
    """
    all_df_magcol = []
    for index,pwv in enumerate(all_pwv):
        atm_bands = the_pc.coll_bandpass_total_nonstd[index]
        mags_pwv1 = {}
        mags_pwv2 = {}
        madus_pwv1 = {}
        madus_pwv2 = {}
        for f in filter_tagnames: 
            mags_pwv1[f] = the_sed1.calc_mag(atm_bands[f])
            mags_pwv2[f] = the_sed2.calc_mag(atm_bands[f])
            madus_pwv1[f] = -2.5*np.log10(the_sed1.calc_adu(atm_bands[f],photoparams))
            madus_pwv2[f] = -2.5*np.log10(the_sed2.calc_adu(atm_bands[f],photoparams))
        # AB colors for star 1
        mags_pwv1["g-r"] = mags_pwv1["g"]- mags_pwv1["r"]  
        mags_pwv1["z-y"] = mags_pwv1["z"]- mags_pwv1["y"] 
        # AB colors for star 2
        mags_pwv2["g-r"] = mags_pwv2["g"]- mags_pwv2["r"]  
        mags_pwv2["z-y"] = mags_pwv2["z"]- mags_pwv2["y"] 
        # ADU mag colors for star 1
        # MADU colors for star 1
        madus_pwv1["g-r"] = madus_pwv1["g"]- madus_pwv1["r"]  
        madus_pwv1["z-y"] = madus_pwv1["z"]- madus_pwv1["y"] 
        # MADU colors for star 2
        madus_pwv2["g-r"] = madus_pwv2["g"]- madus_pwv2["r"]  
        madus_pwv2["z-y"] = madus_pwv2["z"]- madus_pwv2["y"] 
        
        df1 = pd.DataFrame(mags_pwv1, index=["abmag_1"]).T
        df2 = pd.DataFrame(mags_pwv2, index=["abmag_2"]).T
        df3 = pd.DataFrame(madus_pwv1, index=["instmag_1"]).T
        df4 = pd.DataFrame(madus_pwv2, index=["instmag_2"]).T
        df = pd.concat([df1,df2,df3,df4],axis=1)
        all_df_magcol.append(df)
        
    return all_df_magcol

In [ ]:
def ComputeRelativeDiffColor(the_all_df_magcol):
    """
    parameters:
    - the_all_df_magcol : pandas dataframe containing colors for the two SED objects
    return:
    
    """

    df_diffcol = pd.DataFrame(columns=["pwv","Czy1_Czy2_abmag","Czy1_Czy2_instmag"])
    for index,pwv in enumerate(all_pwv):
        the_df = the_all_df_magcol[index]
        DCzyabmag = the_df.loc["z-y","abmag_1"] - the_df.loc["z-y","abmag_2"]
        DCzyadumag = the_df.loc["z-y","instmag_1"] - the_df.loc["z-y","instmag_2"]
        df_diffcol.loc[index] = [pwv,DCzyabmag,DCzyadumag]

    # Relative difference color is in mmag for plotting
    df_diffcol["Czy1_Czy2_abmag_rel"] =  (df_diffcol["Czy1_Czy2_abmag"] - df_diffcol.loc[0,"Czy1_Czy2_abmag"])*1000
    df_diffcol["Czy1_Czy2_instmag_rel"] =  (df_diffcol["Czy1_Czy2_instmag"] - df_diffcol.loc[0,"Czy1_Czy2_instmag"])*1000

    # differentiation  mmag per mm
    dx = np.diff(df_diffcol.pwv.values).mean()
    dy1 = np.diff(df_diffcol["Czy1_Czy2_abmag_rel"].values)
    dy2 = np.diff(df_diffcol["Czy1_Czy2_instmag_rel"].values)
    z_y0 = df_diffcol.loc[0,"Czy1_Czy2_abmag"]
    return df_diffcol

### rubinsimphot

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir


In [ ]:
am0 =1.20    # airmass
pwv0 = 3.0  # Precipitable water vapor vertical column depth in mm
oz0 = 300.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp=1     # Number of aerosol components
tau0= 0.04 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent

In [ ]:
pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

### SED pickles

In [ ]:
# Find the throughputs directory 
#fdir = os.getenv('RUBIN_SIM_DATA_DIR')
fdir = get_data_dir()
if fdir is None:  #environment variable not set
    fdir = os.path.join(os.getenv('HOME'), 'rubin_sim_data')
seddir = os.path.join(fdir, 'pysynphot', 'pickles')
seddir_uvk = os.path.join(seddir,"dat_uvk")
all_pickles_uvk = sorted(os.listdir(seddir_uvk))
file_ref = os.path.join(seddir_uvk, "pickles_uk.fits")
hdul = fits.open(file_ref)
df_pickle = pd.DataFrame(hdul[1].data)

In [ ]:
NSED = len(df_pickle)

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 0.2), layout='constrained')
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=0, vmax=NSED)
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=ax, orientation='horizontal', label='spectral type')

# change the number of labels
labels = [item.get_text() for item in cbar.ax.get_xticklabels()]
cbar.ax.locator_params(axis='x', nbins=4*len(labels)) 
labels = [item.get_text() for item in cbar.ax.get_xticklabels()]

# rename the labels
Nlabels = len(labels)
for ilab in range(0,Nlabels-1):
    the_label = int(labels[ilab])
    labels[ilab] = df_pickle.loc[the_label,"SPTYPE"]
cbar.ax.set_xticklabels(labels,rotation=45);
plt.show()



In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
for index in np.arange(NSED):
    filename = df_pickle.loc[index,"FILENAME"].strip()+".fits"
    fullfilename = os.path.join(seddir_uvk,filename) 
    hdul = fits.open(fullfilename)
    dff = pd.DataFrame(hdul[1].data)
    if index < 100 or index > 104: 
        ax.plot(dff.WAVELENGTH,dff.FLUX,color=all_colors[index])
ax.set_yscale('log')
ax.set_xlim(3000.,11000)
ax.set_ylim(1e-11,1e-6)
ax.grid()
ax.set_title("Pickles $F_\lambda$")
ax.set_ylabel("$F_\lambda$")
ax.set_xlabel("$\lambda \, (\AA$)")
#plt.colorbar(ax.collections[0],ax=ax,orientation = 'horizontal',shrink=0.8)
#ax.collections[-1].colorbar

cbar =fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),ax=ax, orientation='vertical', label='spectral type')

#cbar = plt.colorbar(ax.collections[0],ax=ax)
#cbar.ax.set_ylabel('spectral type', rotation=90)

labels = [item.get_text() for item in cbar.ax.get_yticklabels()]
cbar.ax.locator_params(axis='y', nbins=2*len(labels)) 
labels = [item.get_text() for item in cbar.ax.get_yticklabels()]

Nlabels = len(labels)
for ilab in range(0,Nlabels-1):
    the_label = int(labels[ilab])
    labels[ilab] = df_pickle.loc[the_label,"SPTYPE"]
cbar.ax.set_yticklabels(labels,rotation=0);
plt.tight_layout()


#### Convert Pickles SED in ubin_sim Photmat

In [ ]:
all_sed = []
all_sed_type = []
all_sed_filename = []
zmag = 20.0
for index in np.arange(NSED):
    if index < 100 or index > 104: 
        spectype = df_pickle.loc[index,"SPTYPE"].strip()
        filename = df_pickle.loc[index,"FILENAME"].strip()+".fits"
        fullfilename = os.path.join(seddir_uvk,filename) 
        hdul = fits.open(fullfilename)
        dff = pd.DataFrame(hdul[1].data)
        wl= dff.WAVELENGTH.values
        flux = dff.FLUX.values
        #wavelen (nm)
        #flambda (ergs/cm^2/s/nm)
        the_sed = Sed(wavelen=wl/10., flambda=flux*10., name=spectype)
        flux_norm = the_sed.calc_flux_norm(zmag, pc.bandpass_total_std['r'])
        the_sed.multiply_flux_norm(flux_norm)
        all_sed.append(the_sed) 
        all_sed_type.append(spectype) 
        all_sed_filename.append(filename)
NSED = len(all_sed)

## Simulate multi atmospheres

In [ ]:
am = 1.2
am1 = 1.0
am2 = 1.5
am3 = 2.0
oz = 300
tau= 0.04
beta = 1
all_pwv = np.arange(0,15.,0.5)
NPWV = len(all_pwv)

In [ ]:
pc.CalculateMultiObs(am,all_pwv,oz,tau,beta)

In [ ]:
NOBS = len(all_pwv)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(8,4))

# Figure 1
axs=fig.add_subplot(1,1,1)
for index,pwv in enumerate(all_pwv):
  
    atm_bands = pc.coll_bandpass_total_nonstd[index]
  
    
    label = f"pwv={pwv:.1f}" 

    for f in filter_tagnames: 
        axs.plot(atm_bands[f].wavelen,atm_bands[f].sb,color=all_colors[index],label=label,lw=0.5)
  
   
axs.plot(pc.WL,pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_title("standard and observed transmission for airmass 1.2")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])


plt.tight_layout()
plt.show()



In [ ]:
#all_df_magcol = ComputeColors(pc,the_sed1,the_sed2)
#df_diffcol = ComputeRelativeDiffColor(all_df_magcol)
#z_y0 = df_diffcol.loc[0,"Czy1_Czy2_abmag"]

#dx = np.diff(df_diffcol.pwv.values).mean()
#dy1 = np.diff(df_diffcol["Czy1_Czy2_abmag_rel"].values)
#dy2 = np.diff(df_diffcol["Czy1_Czy2_instmag_rel"].values)

In [ ]:
df_mag_pkl = pd.DataFrame(columns = ["index","filename","spectype",
                                 "magu","magg","magr","magi","magz","magy",
                                 "aduu","adug","adur","adui","aduz","aduy"])
mags_std = {}
adcs_stdmag = {}

for index in np.arange(NSED):
    spectype = all_sed_type[index]
    filename = all_sed_filename[index]
    the_sed = all_sed[index] 
    for index2,f in enumerate(filter_tagnames) :
        mags_std[f] = the_sed.calc_mag(pc.bandpass_total_std[f])
        adcs_stdmag[f] = -2.5*np.log10(the_sed.calc_adu(pc.bandpass_total_std[f],photoparams))
    df_mag_pkl.loc[index] = [ index,filename, spectype ,
                    mags_std["u"],mags_std["g"],mags_std["r"],mags_std["i"],mags_std["z"],mags_std["y"],
                    adcs_stdmag["u"],adcs_stdmag["g"],adcs_stdmag["r"],adcs_stdmag["i"],adcs_stdmag["z"],adcs_stdmag["y"]]
                                
df_mag_pkl["magG_R"] = df_mag_pkl["magg"] - df_mag_pkl["magr"]
df_mag_pkl["magZ_Y"] = df_mag_pkl["magz"] - df_mag_pkl["magy"]
df_mag_pkl["aduG_R"] = df_mag_pkl["adug"] - df_mag_pkl["adur"]
df_mag_pkl["aduZ_Y"] = df_mag_pkl["aduz"] - df_mag_pkl["aduy"]


In [ ]:
 df_mag_pkl

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,8))

df_mag_pkl.plot.scatter(x="aduZ_Y",y="aduG_R",marker="+",c="index",s=60,colormap="jet",ax=ax,grid=True,colorbar=False,label="Pickles")
#cbar = plt.colorbar(ax.collections[0],ax=ax,orientation = 'horizontal',shrink=0.8)
cbar = plt.colorbar(ax.collections[0],ax=ax)
cbar.ax.set_ylabel('spectral type', rotation=90)

labels = [item.get_text() for item in cbar.ax.get_yticklabels()]
cbar.ax.locator_params(axis='y', nbins=2*len(labels)) 
labels = [item.get_text() for item in cbar.ax.get_yticklabels()]

Nlabels = len(labels)

for ilab in range(0,Nlabels-1):
    the_label = int(labels[ilab])
    labels[ilab] = df_mag_pkl.loc[the_label,"spectype"]
cbar.ax.set_yticklabels(labels,rotation=0);
ax.set_title("Color-Color plot (obs ADU mag) of Pickles stars")
ax.set_xlabel("Z-Y (adu) (mag)")
ax.set_ylabel("G-R (adu) (mag)")
ax.set_aspect("equal")
ax.errorbar(df_data["z_y_ap"],df_data["g_r_ap"],xerr=df_data["sig_z_y_ap"] , yerr=df_data["sig_g_r_ap"], fmt="o",color='k',ecolor="grey",label="Auxtel data (Aperture flux)")


ax.legend()


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,8))

df_mag_pkl.plot.scatter(x="magZ_Y",y="magG_R",marker="+",c="index",s=60,colormap="jet",ax=ax,grid=True,colorbar=False,label="Pickles")
#cbar = plt.colorbar(ax.collections[0],ax=ax,orientation = 'horizontal',shrink=0.8)
cbar = plt.colorbar(ax.collections[0],ax=ax)
cbar.ax.set_ylabel('spectral type', rotation=90)

labels = [item.get_text() for item in cbar.ax.get_yticklabels()]
cbar.ax.locator_params(axis='y', nbins=2*len(labels)) 
labels = [item.get_text() for item in cbar.ax.get_yticklabels()]

Nlabels = len(labels)

for ilab in range(0,Nlabels-1):
    the_label = int(labels[ilab])
    labels[ilab] = df_mag_pkl.loc[the_label,"spectype"]
cbar.ax.set_yticklabels(labels,rotation=0);
ax.set_title("Color-Color plot (AB mag) of Pickles stars")
ax.set_xlabel("Z-Y (AB) (mag)")
ax.set_ylabel("G-R (AB) (mag)")
ax.set_aspect("equal")
ax.errorbar(df_data["z_y_psf"],df_data["g_r_psf"],xerr=df_data["sig_z_y_psf"] , yerr=df_data["sig_g_r_psf"], fmt="o",color='k',ecolor="grey",label="Auxtel data (PSF flux)")

ax.legend()


## Extract the Neighbourgs and Data for NearestNeighbourg

In [ ]:
# neighbourgs
df_colors = df_mag_pkl[["index","spectype","magG_R","magZ_Y"]]
x_pkl = df_colors["magZ_Y"].values
y_pkl = df_colors["magG_R"].values
X_pkl = list(zip(x_pkl,y_pkl))

In [ ]:
# data
x_dat = df_data["z_y_psf"].values
y_dat = df_data["g_r_psf"].values
X_data = list(zip(x_dat,y_dat))


In [ ]:
indices1,distances1 = FindNearestNeighbors(X_data,X_pkl)
indices2,distances2 = FindNearestKDT(X_data,X_pkl)
indices3,distances3 = FindNearestBallT(X_data,X_pkl)

In [ ]:
indices_flat = indices1.flatten()

In [ ]:
#np.hstack([indices1,indices2,indices3])

In [ ]:
Ndata = len(df_data)
photom_wl = np.array([FILTERWL[1,2],FILTERWL[2,2],FILTERWL[3,2],FILTERWL[4,2],FILTERWL[5,2]])

In [ ]:
df_matchobjsed = pd.DataFrame(columns=["objnum","sedfnname","sedtype"])

for idx_data in range(Ndata):
    row  = df_data.iloc[idx_data]
    objnum = int(row.obj)
    idx_sed = indices_flat[idx_data]   
    sed_type = all_sed_type[idx_sed]
    sed_filename = all_sed_filename[idx_sed]
    df_matchobjsed.loc[idx_data] = [objnum,sed_filename,sed_type]  
    

In [ ]:
df_matchobjsed

In [ ]:
for idx_data in range(Ndata):
    fig,(ax,ax2) = plt.subplots(1,2,figsize=(16,6))
    
    #get the data
    row  = df_data.iloc[idx_data]
    objnum = int(row.obj)
    label_data = f"object {objnum}"
    mags = np.array([row["g_psf"],row["r_psf"],row["i_psf"],row["z_psf"], row["y_psf"]])
    mags_err = np.array([row["sg_psf"],row["sr_psf"],row["si_psf"],row["sz_psf"], row["sy_psf"]])
    
    
    ### Left plot

    # plot data
    ax.errorbar(photom_wl,mags,yerr=mags_err,fmt=".",color="k",ecolor="grey",elinewidth=1.)  
    ax.scatter(photom_wl,mags,marker="o",color="k",s=20,label=label_data) 
    
    ax.set_xlim(300.,1100.)
    
    
    # get the SED
   
    idx_sed = indices_flat[idx_data]
    the_sed = all_sed[idx_sed]
    flux_norm = the_sed.calc_flux_norm(row["r_psf"], pc.bandpass_total_std['r'])
    the_sed.multiply_flux_norm(flux_norm)
    name_sed = the_sed.name
    sed_type = all_sed_type[idx_sed]
    
    assert name_sed == sed_type
    sed_filename = all_sed_filename[idx_sed]
        
    mags_pkl = {}
    
    for f in ["g","r","i","z","y"] :
        mags_pkl[f] = the_sed.calc_mag(pc.bandpass_total_std[f])
        
    all_mag_pkl = np.array([mags_pkl["g"],mags_pkl["r"],mags_pkl["i"],mags_pkl["z"],mags_pkl["y"]])    
    label_simmag = f"calc AB mag for pickle {name_sed}"
    
    ax.scatter(photom_wl,all_mag_pkl,marker = "o",color="r",label=label_simmag)
    
    
    label_sed = f"pickle spectrum sed_fn : {sed_filename}"
    ax.plot(the_sed.wavelen,-2.5*np.log10(the_sed.fnu/F0),"b:",label=label_sed)
    
    ax.legend()
    
    ax.set_xlabel("$\lambda \, (nm)$")
    ax.set_ylabel("Magnitude (AB-mag)")
    ax.yaxis.set_inverted(True)
    ax.set_ylim(20.,12.)
   
    
   

    ## Right plot
    
    # data point : convert mags in fnu then in flambda
    all_fnudata_points = F0*np.power(10,-0.4*mags)
    data_sed_points =  Sed(wavelen=photom_wl,fnu=all_fnudata_points)
    data_flux_flambda = data_sed_points.flambda
    data_flux_flambda_err =  data_flux_flambda*mags_err*np.log(10)/2.5 
    
    ax2.errorbar(photom_wl, data_flux_flambda,yerr=data_flux_flambda_err,fmt=".",color="k",ecolor="grey",elinewidth=1.)  
    ax2.scatter(photom_wl,  data_flux_flambda ,marker = "o",color="k",label=label_data)
    
    
    # pickle points : convert mags in fnu then in flambda
    all_fnusim_points = F0*np.power(10,-0.4*all_mag_pkl)
    sim_sed_points = Sed(wavelen=photom_wl,fnu=all_fnusim_points)
    label_simmag = f"band average $f_\lambda$ for pickle {name_sed}"
    ax2.scatter(photom_wl, sim_sed_points.flambda,marker = "o",color="r",label=label_simmag)
    
    ax2.plot(the_sed.wavelen,the_sed.flambda,"b:",label=label_sed)
    
    ax2.set_xlim(300.,1100.)
    ax2.set_xlabel("$\lambda \, (nm)$")
    ax2.set_ylabel("$f_\lambda \,\; (ergs/cm^2/s/nm)$")
    ax2.legend()
    
    plt.suptitle(f"Flux comparison data/pickle for object {objnum}",y=.95,fontsize=16)
    
    plt.tight_layout()
    plt.show()

## Save

In [ ]:
df_matchobjsed

In [ ]:
df_matchobjsed.to_csv(filename_catalogmatching) 